In [21]:
import pandas as pd

df = pd.read_csv("APsrtc_bus_details1.csv")
df = pd.read_csv("Astc_bus_details.csv")
df = pd.read_csv("Bsrtc_bus_details.csv")
df = pd.read_csv("Hrtc_bus_details.csv")
df = pd.read_csv("ksrtc_bus_details1.csv")
df = pd.read_csv("Rsrtc_bus_details.csv")
df = pd.read_csv("SBsrtc_bus_details.csv")
df = pd.read_csv("Tsrtc_bus_details.csv")
df = pd.read_csv("UPsrtc_bus_details.csv")
df = pd.read_csv("WBtc_bus_details.csv")
df = pd.read_csv("Psrtc_bus_details.csv")
df = pd.read_csv("CTU_bus_details.csv")

# List of CSV file paths and corresponding state names
csv_files = {
    "APsrtc_bus_details1.csv": "Andhra Pradesh",
    "Astc_bus_details.csv": "Assam",
    "Bsrtc_bus_details.csv": "Bihar",
    "Hrtc_bus_details.csv": "Himachal Pradesh",
    "ksrtc_bus_details1.csv": "Kerala",
    "Rsrtc_bus_details.csv": "Rajasthan",
    "SBsrtc_bus_details.csv": "South Bengal",
    "Tsrtc_bus_details.csv": "Telangana",
    "UPsrtc_bus_details.csv": "Uttar Pradesh",
    "WBtc_bus_details.csv": "West Bengal",
    "Psrtc_bus_details.csv": "Punjab",
    "CTU_bus_details.csv": "Chandigarh"
}

# Read each CSV file into a DataFrame, add the new column, and store it in a list
dfs = []
for file, state in csv_files.items():
    df = pd.read_csv(file)
    df['states'] = state
    dfs.append(df)

# Concatenate all DataFrames in the list
combined_df = pd.concat(dfs, ignore_index=True)

# Save the updated combined DataFrame to a new CSV file
combined_df.to_csv("RB_bus_details1.csv", index=False)

In [7]:
import pandas as pd
import pymysql

# Database connection details
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWORD = 'Abcd1234'
DB_NAME = 'redbus'

# CSV file path
CSV_FILE = 'RB_bus_details1.csv'

def load_data_to_mysql():
    # Load and clean CSV data
    try:
        df = pd.read_csv(CSV_FILE)
        print("CSV data loaded successfully!")
        print("CSV Columns:", df.columns.tolist())  # Debugging step
        
        # Correct column names
        df.rename(columns={'Bus_Name': 'BusName', 'Bus_Type': 'BusType'}, inplace=True)

        # Fill missing values for BusName and BusType
        
        df['BusName'] = df['BusName'].fillna("Unknown")
        df['BusType'] = df['BusType'].fillna("Unknown")

        
        # Clean other data
        df['Seat_Availability'] = df['Seat_Availability'].str.extract(r'(\d+)').astype(float)
        df['Star_Rating'] = pd.to_numeric(df['Star_Rating'], errors='coerce')
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
        df.fillna({'Seat_Availability': 0, 'Star_Rating': 0, 'Price': 0}, inplace=True)
        df = df.replace({pd.NA: None, pd.NaT: None})  # Replace NaN, NA, NaT with None
        
    except Exception as e:
        print(f"Error loading or processing CSV: {e}")
        return

    # Connect to MySQL
    try:
        connection = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME
        )
        cursor = connection.cursor()
        print("Connected to MySQL database.")
    except pymysql.MySQLError as err:
        print(f"Error connecting to MySQL: {err}")
        return

    # Create the table if it doesn't exist
    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME}")
        cursor.execute(f"USE {DB_NAME}")
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS Bus_routes (
            id INT AUTO_INCREMENT PRIMARY KEY,
            route_name TEXT,
            route_link TEXT,
            busname TEXT,
            bustype TEXT,
            departing_time TIME,
            duration TEXT,
            reaching_time TIME,
            star_rating FLOAT,
            price DECIMAL(10, 2),
            seats_available INT,
            states TEXT
        )
        """)
        print("Table `bus_routes` created successfully!")
    except pymysql.MySQLError as err:
        print(f"Error creating table: {err}")
        return

    # Insert data into the table
    try:
        for _, row in df.iterrows():
            sql = """
            INSERT INTO Bus_routes (
                route_name, route_link, busname, bustype,
                departing_time, duration, reaching_time,
                star_rating, price, seats_available,states
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            data = (
                row.get('Route_Name', None),
                row.get('Route_Link', None),
                row.get('BusName', None),
                row.get('BusType', None),
                row.get('Departing_Time', None),
                row.get('Duration', None),
                row.get('Reaching_Time', None),
                row.get('Star_Rating', None),
                row.get('Price', None),
                row.get('Seat_Availability', None),
                row.get('states', None)
            )
                       
            cursor.execute(sql, data)
        connection.commit()
        print(f"Data inserted successfully into `{DB_NAME}.Bus_routes`.")
    except pymysql.MySQLError as err:
        print(f"Error inserting data: {err}")
    finally:
        cursor.close()
        connection.close()

if __name__ == "__main__":
    load_data_to_mysql()


CSV data loaded successfully!
CSV Columns: ['Route_Name', 'Route_Link', 'Bus_Name', 'Bus_Type', 'Departing_Time', 'Duration', 'Reaching_Time', 'Star_Rating', 'Price', 'Seat_Availability', 'states']
Connected to MySQL database.
Table `bus_routes` created successfully!
Data inserted successfully into `redbus.Bus_routes`.


In [9]:
import pymysql

# Define database credentials
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWORD = 'Abcd1234'
DB_NAME = 'redbus'

try:
    connection = pymysql.connect(
        host=DB_HOST,  # Use the DB_HOST variable
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME
    )
    cursor = connection.cursor()
    print("Connected to MySQL database.")
except pymysql.MySQLError as err:
    print(f"Error connecting to MySQL: {err}")


Connected to MySQL database.


In [13]:
cursor.execute("DESCRIBE redbus.Bus_routes")
for i in cursor:
    print(i)

('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('route_name', 'text', 'YES', '', None, '')
('route_link', 'text', 'YES', '', None, '')
('busname', 'text', 'YES', '', None, '')
('bustype', 'text', 'YES', '', None, '')
('departing_time', 'time', 'YES', '', None, '')
('duration', 'text', 'YES', '', None, '')
('reaching_time', 'time', 'YES', '', None, '')
('star_rating', 'float', 'YES', '', None, '')
('price', 'decimal(10,2)', 'YES', '', None, '')
('seats_available', 'int', 'YES', '', None, '')
('states', 'text', 'YES', '', None, '')


In [15]:
cursor.execute("sELECT * from redbus.Bus_routes limit  5")
for i in cursor:
    print(i)

(1, 'Kakinada to Visakhapatnam', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'APSRTC - 2538', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=900), '04h 00m', datetime.timedelta(seconds=15300), 3.8, Decimal('281.00'), 34, 'Andhra Pradesh')
(2, 'Kakinada to Visakhapatnam', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'APSRTC - 2837', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=1800), '03h 00m', datetime.timedelta(seconds=12600), 4.6, Decimal('281.00'), 33, 'Andhra Pradesh')
(3, 'Kakinada to Visakhapatnam', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'APSRTC - 23551', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=2700), '04h 00m', datetime.timedelta(seconds=17100), 4.1, Decimal('281.00'), 34, 'Andhra Pradesh')
(4, 'Kakinada to Visakhapatnam', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'APSRTC - 2536', 'INDRA(A.C. Seater)', datetime.timedelta(sec

In [17]:
cursor.execute("sELECT Route_name from redbus.Bus_routes limit 5")
for i in cursor:
    print(i)

('Kakinada to Visakhapatnam',)
('Kakinada to Visakhapatnam',)
('Kakinada to Visakhapatnam',)
('Kakinada to Visakhapatnam',)
('Kakinada to Visakhapatnam',)


In [19]:
cursor.execute("SELECT Route_Name, Duration FROM redbus.bus_routes ORDER BY Duration DESC LIMIT 5")
for i in cursor:
    print(i)

('Chittoor (Andhra Pradesh) to Bangalore', '28h 25m')
('Chennai to Tirupati', '27h 30m')
('Karimnagar to Hyderabad', '21h 45m')
('Delhi to Gorakhpur (uttar pradesh)', '21h 34m')
('Agra to Motihari', '19h 45m')


In [21]:
cursor.execute("SELECT Route_Name, Duration FROM redbus.bus_routes ORDER BY Duration ASC LIMIT 5")
for i in cursor:
    print(i)

('Peddapalli (Telangana) to Hyderabad', '0-19h 0-25m')
('Karimnagar to Hyderabad', '0-20h 0-29m')
('Agra to Motihari', '0-6h 00m')
('Kolkata to Ramnagar (West Bengal)', '00h 15m')
('Kolkata to Heria', '00h 15m')


In [23]:
cursor.execute("SELECT Route_Name, MAX(Price) AS Highest_Price FROM redbus.bus_routes GROUP BY Route_Name limit 10")
for i in cursor:
    print(i)

('Kakinada to Visakhapatnam', Decimal('5000.00'))
('Chittoor (Andhra Pradesh) to Bangalore', Decimal('1180.00'))
('Tirupati to Bangalore', Decimal('900.00'))
('Hyderabad to Ongole', Decimal('1200.00'))
('Ongole to Hyderabad', Decimal('2000.00'))
('Visakhapatnam to Kakinada', Decimal('5000.00'))
('Kadiri to Bangalore', Decimal('3999.00'))
('Vinukonda to Hyderabad', Decimal('4000.00'))
('Bangalore to Kadiri', Decimal('3999.00'))
('Hyderabad to Narasaraopet', Decimal('5000.00'))
